In [5]:
import os, sys, time, glob, random, argparse
import numpy as np
from copy import deepcopy
import torch
import torch.nn as nn
from pathlib import Path
sys.path.append(str('/home/taoliu/Data/NAS-Projects-master/lib'))
sys.path.append(str('/home/taoliu/Data/NAS-Projects-master/configs'))
from config_utils import load_config, dict2config, configure2str
from datasets     import get_datasets, SearchDataset
from procedures   import prepare_seed, prepare_logger, save_checkpoint, copy_checkpoint, get_optim_scheduler
from utils        import get_model_infos, obtain_accuracy
from log_utils    import AverageMeter, time_string, convert_secs2time
from models       import get_cell_based_tiny_net, get_search_spaces
from nas_102_api  import NASBench102API as API

In [6]:
parser = argparse.ArgumentParser("Random search for NAS.")
parser.add_argument('--data_path',          type=str,   default= '/data/taoliu/ENNAS_benchmark102/data/cifar-10-batches-py',    help='Path to dataset')
parser.add_argument('--dataset',            type=str,   default= 'cifar10',choices=['cifar10', 'cifar100', 'ImageNet16-120'], help='Choose between Cifar10/100 and ImageNet-16.')
# channels and number-of-cells
parser.add_argument('--search_space_name',  type=str,   default= 'nas-bench-102', help='The search space name.')
parser.add_argument('--config_path',        type=str,   default= '/home/taoliu/Data/NAS-Projects-master/configs/nas-benchmark/algos/RANDOM.config'  ,help='The path to the configuration.')
parser.add_argument('--max_nodes',          type=int,   default= 4 , help='The maximum number of nodes.')
parser.add_argument('--channel',            type=int,   default= 16 , help='The number of channels.')
parser.add_argument('--num_cells',          type=int,   default= 5, help='The number of cells in one stage.')
parser.add_argument('--select_num',         type=int,   default= 100, help='The number of selected architectures to evaluate.')
# log
parser.add_argument('--workers',            type=int,   default=2, help='number of data loading workers (default: 2)')
parser.add_argument('--save_dir',           type=str,   default='./RANDOM_NSAS_G_0.2_seed0/output/search-cell-nas-bench-102-cifar10',help='Folder to save checkpoints and log.')
parser.add_argument('--arch_nas_dataset',   type=str,   default='/home/taoliu/Data/NAS-Projects-master/NAS-Bench-102-v1_0-e61699.pth',help='The path to load the architecture dataset (tiny-nas-benchmark).')
parser.add_argument('--print_freq',         type=int,   default=200,help='print frequency (default: 200)')
parser.add_argument('--rand_seed',          type=int,   default=0,help='manual seed')
args = parser.parse_args([])

In [7]:
def cal_compensate(pre_index,cur_index,pre_hardwts,cur_hardwts):
    compen_index   = cur_index
    for i in range(pre_index.shape[0]):
        if pre_index[i][0].item()==cur_index[i][0].item():
            a=np.random.randint(5)
            while a==pre_index[i][0].item():
                a=np.random.randint(5)
            compen_index[i][0]=a              
    one_h   = torch.zeros_like(cur_hardwts).scatter_(-1, compen_index, 1.0)
    compen_hardwts = one_h - cur_hardwts.detach() + cur_hardwts
    
    return compen_index,compen_hardwts

def random_gene(cur_index,cur_hardwts):
    compen_index   = cur_index
    for i in range(cur_index.shape[0]):        
        a=np.random.randint(5)
        compen_index[i][0]=a              
    one_h   = torch.zeros_like(cur_hardwts).scatter_(-1, compen_index, 1.0)
    compen_hardwts = one_h - cur_hardwts.detach() + cur_hardwts
    
    return compen_index,compen_hardwts


def diver_gene(cur_index,cur_hardwts):
    compen_index   = cur_index
    for i in range(cur_index.shape[0]):
        a=np.random.randint(5)
        while a==cur_index[i][0].item():
            a=np.random.randint(5)
        compen_index[i][0]=a              
    one_h   = torch.zeros_like(cur_hardwts).scatter_(-1, compen_index, 1.0)
    compen_hardwts = one_h - cur_hardwts.detach() + cur_hardwts
    
    return compen_index,compen_hardwts




def search_func(xloader, network, criterion, scheduler, w_optimizer, epoch_str, print_freq, logger):
    data_time, batch_time = AverageMeter(), AverageMeter()
    base_losses, base_top1, base_top5 = AverageMeter(), AverageMeter(), AverageMeter()
    network.train()
    end = time.time()
    arch_pre,genotypes_pre,operations_pre=network.module.random_genotype( False )
    for step, (base_inputs, base_targets, arch_inputs, arch_targets) in enumerate(xloader):
        scheduler.update(None, 1.0 * step / len(xloader))
        base_targets = base_targets.cuda(non_blocking=True)
        arch_targets = arch_targets.cuda(non_blocking=True)
        # measure data loading time
        data_time.update(time.time() - end)
    
        # update the weights
        arch,genotypes,operations=network.module.random_genotype( True )
        
                
        w_optimizer.zero_grad()
        _, logits = network(base_inputs)
        loss = criterion(logits, base_targets)
        
        arch_random,genotypes_random,operations_random=network.module.random_genotype( False ) 
        arch_random=network.module.set_genotype(arch_random)
        _, logits = network(base_inputs)
        random_loss = criterion(logits, base_targets) 
        
        
        arch_diver,genotypes_diver,operations_diver= network.module.arch_compensate(operations_random,operations)
        
        arch_diver=network.module.set_genotype(arch_diver)
        _, logits = network(base_inputs)
        diver_loss = criterion(logits, base_targets)        
         
        
        arch=network.module.set_genotype(arch)
        
        
        base_loss=0.2*loss+0.4*random_loss+0.4*diver_loss
        
        base_loss.backward()
        nn.utils.clip_grad_norm_(network.parameters(), 5)
        w_optimizer.step()
            # record
        base_prec1, base_prec5 = obtain_accuracy(logits.data, base_targets.data, topk=(1, 5))
        base_losses.update(base_loss.item(),  base_inputs.size(0))
        base_top1.update  (base_prec1.item(), base_inputs.size(0))
        base_top5.update  (base_prec5.item(), base_inputs.size(0))

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()
        
        
        #####current to pre
        arch_pre,genotypes_pre,operations_pre=arch,genotypes,operations
        if step % print_freq == 0 or step + 1 == len(xloader):
            Sstr = '*SEARCH* ' + time_string() + ' [{:}][{:03d}/{:03d}]'.format(epoch_str, step, len(xloader))
            Tstr = 'Time {batch_time.val:.2f} ({batch_time.avg:.2f}) Data {data_time.val:.2f} ({data_time.avg:.2f})'.format(batch_time=batch_time, data_time=data_time)
            Wstr = 'Base [Loss {loss.val:.3f} ({loss.avg:.3f})  Prec@1 {top1.val:.2f} ({top1.avg:.2f}) Prec@5 {top5.val:.2f} ({top5.avg:.2f})]'.format(loss=base_losses, top1=base_top1, top5=base_top5)
            logger.log(Sstr + ' ' + Tstr + ' ' + Wstr)
    return base_losses.avg, base_top1.avg, base_top5.avg


def valid_func(xloader, network, criterion):
    data_time, batch_time = AverageMeter(), AverageMeter()
    arch_losses, arch_top1, arch_top5 = AverageMeter(), AverageMeter(), AverageMeter()
    network.eval()
    end = time.time()
    with torch.no_grad():
        for step, (arch_inputs, arch_targets) in enumerate(xloader):
            arch_targets = arch_targets.cuda(non_blocking=True)
          # measure data loading time
            data_time.update(time.time() - end)
       # prediction

            network.module.random_genotype( True )
            _, logits = network(arch_inputs)
            arch_loss = criterion(logits, arch_targets)
      # record
            arch_prec1, arch_prec5 = obtain_accuracy(logits.data, arch_targets.data, topk=(1, 5))
            arch_losses.update(arch_loss.item(),  arch_inputs.size(0))
            arch_top1.update  (arch_prec1.item(), arch_inputs.size(0))
            arch_top5.update  (arch_prec5.item(), arch_inputs.size(0))
      # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()
    return arch_losses.avg, arch_top1.avg, arch_top5.avg

def search_find_best(valid_loader, network, criterion, select_num):
    best_acc = 0
    for iarch in range(select_num):
        arch, genotypes,operations = network.module.random_genotype( True )
        valid_a_loss, valid_a_top1, valid_a_top5  = valid_func(valid_loader, network, criterion)
        if best_acc < valid_a_top1:
            best_arch, best_acc = arch, valid_a_top1
    return best_arch

In [8]:
#def main(xargs):
xargs=args
assert torch.cuda.is_available(), 'CUDA is not available.'
torch.backends.cudnn.enabled   = True
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True
torch.set_num_threads( xargs.workers )
prepare_seed(xargs.rand_seed)
logger = prepare_logger(args)

train_data, valid_data, xshape, class_num = get_datasets(xargs.dataset, xargs.data_path, -1)
if xargs.dataset == 'cifar10' or xargs.dataset == 'cifar100':
    split_Fpath = '/home/taoliu/Data/NAS-Projects-master/configs/nas-benchmark/cifar-split.txt'
    cifar_split = load_config(split_Fpath, None, None)
    train_split, valid_split = cifar_split.train, cifar_split.valid
    logger.log('Load split file from {:}'.format(split_Fpath))
#elif xargs.dataset.startswith('ImageNet16'):
#  # all_indexes = list(range(len(train_data))) ; random.seed(111) ; random.shuffle(all_indexes)
#  # train_split, valid_split = sorted(all_indexes[: len(train_data)//2]), sorted(all_indexes[len(train_data)//2 :])
#  # imagenet16_split = dict2config({'train': train_split, 'valid': valid_split}, None)
#  # _ = configure2str(imagenet16_split, 'temp.txt')
#  split_Fpath = 'configs/nas-benchmark/{:}-split.txt'.format(xargs.dataset)
#  imagenet16_split = load_config(split_Fpath, None, None)
#  train_split, valid_split = imagenet16_split.train, imagenet16_split.valid
#  logger.log('Load split file from {:}'.format(split_Fpath))
else:
    raise ValueError('invalid dataset : {:}'.format(xargs.dataset))
config = load_config(xargs.config_path, {'class_num': class_num, 'xshape': xshape}, logger)
logger.log('config : {:}'.format(config))
# To split data
train_data_v2 = deepcopy(train_data)
train_data_v2.transform = valid_data.transform
valid_data    = train_data_v2
search_data   = SearchDataset(xargs.dataset, train_data, train_split, valid_split)
# data loader
search_loader = torch.utils.data.DataLoader(search_data, batch_size=config.batch_size, shuffle=True , num_workers=xargs.workers, pin_memory=True)
valid_loader  = torch.utils.data.DataLoader(valid_data, batch_size=config.batch_size, sampler=torch.utils.data.sampler.SubsetRandomSampler(valid_split), num_workers=xargs.workers, pin_memory=True)
logger.log('||||||| {:10s} ||||||| Search-Loader-Num={:}, Valid-Loader-Num={:}, batch size={:}'.format(xargs.dataset, len(search_loader), len(valid_loader), config.batch_size))
logger.log('||||||| {:10s} ||||||| Config={:}'.format(xargs.dataset, config))

search_space = get_search_spaces('cell', xargs.search_space_name)
model_config = dict2config({'name': 'RANDOM', 'C': xargs.channel, 'N': xargs.num_cells,
                          'max_nodes': xargs.max_nodes, 'num_classes': class_num,
                          'space'    : search_space}, None)
search_model = get_cell_based_tiny_net(model_config)

w_optimizer, w_scheduler, criterion = get_optim_scheduler(search_model.parameters(), config)
logger.log('w-optimizer : {:}'.format(w_optimizer))
logger.log('w-scheduler : {:}'.format(w_scheduler))
logger.log('criterion   : {:}'.format(criterion))
if xargs.arch_nas_dataset is None: api = None
else                             : api = API(xargs.arch_nas_dataset)
logger.log('{:} create API = {:} done'.format(time_string(), api))

last_info, model_base_path, model_best_path = logger.path('info'), logger.path('model'), logger.path('best')
network, criterion = torch.nn.DataParallel(search_model,device_ids=[0]).cuda(), criterion.cuda()

if last_info.exists(): # automatically resume from previous checkpoint
    logger.log("=> loading checkpoint of the last-info '{:}' start".format(last_info))
    last_info   = torch.load(last_info)
    start_epoch = last_info['epoch']
    checkpoint  = torch.load(last_info['last_checkpoint'])
    genotypes   = checkpoint['genotypes']
    valid_accuracies = checkpoint['valid_accuracies']
    search_model.load_state_dict( checkpoint['search_model'] )
    w_scheduler.load_state_dict ( checkpoint['w_scheduler'] )
    w_optimizer.load_state_dict ( checkpoint['w_optimizer'] )
    logger.log("=> loading checkpoint of the last-info '{:}' start with {:}-th epoch.".format(last_info, start_epoch))
else:
    logger.log("=> do not find the last-info file : {:}".format(last_info))
    start_epoch, valid_accuracies, genotypes = 0, {'best': -1}, {}

# start training
start_time, search_time, epoch_time, total_epoch = time.time(), AverageMeter(), AverageMeter(), config.epochs + config.warmup
for epoch in range(start_epoch, total_epoch):
    w_scheduler.update(epoch, 0.0)
    need_time = 'Time Left: {:}'.format( convert_secs2time(epoch_time.val * (total_epoch-epoch), True) )
    epoch_str = '{:03d}-{:03d}'.format(epoch, total_epoch)
    logger.log('\n[Search the {:}-th epoch] {:}, LR={:}'.format(epoch_str, need_time, min(w_scheduler.get_lr())))

# selected_arch = search_find_best(valid_loader, network, criterion, xargs.select_num)
    search_w_loss, search_w_top1, search_w_top5 = search_func(search_loader, network, criterion, w_scheduler, w_optimizer, epoch_str, xargs.print_freq, logger)
    search_time.update(time.time() - start_time)
    logger.log('[{:}] searching : loss={:.2f}, accuracy@1={:.2f}%, accuracy@5={:.2f}%, time-cost={:.1f} s'.format(epoch_str, search_w_loss, search_w_top1, search_w_top5, search_time.sum))
    valid_a_loss , valid_a_top1 , valid_a_top5  = valid_func(valid_loader, network, criterion)
    logger.log('[{:}] evaluate  : loss={:.2f}, accuracy@1={:.2f}%, accuracy@5={:.2f}%'.format(epoch_str, valid_a_loss, valid_a_top1, valid_a_top5))
    #cur_arch = search_find_best(valid_loader, network, criterion, xargs.select_num)
    cur_arch = search_find_best(valid_loader, network, criterion, 2)
    genotypes[epoch] = cur_arch
# check the best accuracy
    valid_accuracies[epoch] = valid_a_top1
    if valid_a_top1 > valid_accuracies['best']:
        valid_accuracies['best'] = valid_a_top1
        find_best = True
    else: find_best = False

# save checkpoint
    save_path = save_checkpoint({'epoch' : epoch + 1,
            'args'  : deepcopy(xargs),
            'search_model': search_model.state_dict(),
            'w_optimizer' : w_optimizer.state_dict(),
            'w_scheduler' : w_scheduler.state_dict(),
            'genotypes'   : genotypes,
            'valid_accuracies' : valid_accuracies},
            model_base_path, logger)
    last_info = save_checkpoint({
      'epoch': epoch + 1,
      'args' : deepcopy(args),
      'last_checkpoint': save_path,
      }, logger.path('info'), logger)
    if find_best:
        logger.log('<<<--->>> The {:}-th epoch : find the highest validation accuracy : {:.2f}%.'.format(epoch_str, valid_a_top1))
        copy_checkpoint(model_base_path, model_best_path, logger)
    if api is not None: logger.log('{:}'.format(api.query_by_arch( genotypes[epoch] )))
# measure elapsed time
    epoch_time.update(time.time() - start_time)
    start_time = time.time()

logger.log('\n' + '-'*200)
logger.log('Pre-searching costs {:.1f} s'.format(search_time.sum))
start_time = time.time()
best_arch, best_acc = None, -1
for iarch in range(xargs.select_num):
    arch = search_model.random_genotype( True )
    valid_a_loss, valid_a_top1, valid_a_top5  = valid_func(valid_loader, network, criterion)
    logger.log('final evaluation [{:02d}/{:02d}] : {:} : accuracy={:.2f}%, loss={:.3f}'.format(iarch, xargs.select_num, arch, valid_a_top1, valid_a_loss))
    if best_arch is None or best_acc < valid_a_top1:
        best_arch, best_acc = arch, valid_a_top1
search_time.update(time.time() - start_time)
logger.log('RANDOM-NAS finds the best one : {:} with accuracy={:.2f}%, with {:.1f} s.'.format(best_arch, best_acc, search_time.sum))
if api is not None: logger.log('{:}'.format( api.query_by_arch(best_arch[0]) ))
logger.close()

Main Function with logger : Logger(dir=RANDOM_NSAS_G_0.2_seed0/output/search-cell-nas-bench-102-cifar10, use-tf=False, writer=None)
Arguments : -------------------------------
arch_nas_dataset : /home/taoliu/Data/NAS-Projects-master/NAS-Bench-102-v1_0-e61699.pth
channel          : 16
config_path      : /home/taoliu/Data/NAS-Projects-master/configs/nas-benchmark/algos/RANDOM.config
data_path        : /data/taoliu/ENNAS_benchmark102/data/cifar-10-batches-py
dataset          : cifar10
max_nodes        : 4
num_cells        : 5
print_freq       : 200
rand_seed        : 0
save_dir         : ./RANDOM_NSAS_G_0.2_seed0/output/search-cell-nas-bench-102-cifar10
search_space_name : nas-bench-102
select_num       : 100
workers          : 2
Python  Version  : 3.6.5 |Anaconda, Inc.| (default, Apr 29 2018, 16:14:56)  [GCC 7.2.0]
Pillow  Version  : 5.1.0
PyTorch Version  : 0.4.1
cuDNN   Version  : 7102
CUDA available   : True
CUDA GPU numbers : 2
CUDA_VISIBLE_DEVICES : None
Files already downloaded and

OSError: [Errno 12] Cannot allocate memory